In [60]:
import numpy as np
import random
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.linear_model import SGDClassifier
from sklearn.datasets.samples_generator import make_blobs

# se crean n puntos separables 
n = 500
state = int(random.randrange(100))
X, y = make_blobs(n_samples=n, centers=2, random_state=state, cluster_std=4.0)


# clasificación a partir de fit lineal 
clf = SGDClassifier(loss="hinge", alpha=0.01, n_iter=100, fit_intercept=True)
clf.fit(X, y)

# Se hacen los putos de la línea que separa los dos grupos y se grafican los puntos y la línea
xx = np.linspace(-20, 20, 3)
yy = np.linspace(-20, 20, 3)

X1, X2 = np.meshgrid(xx, yy)
Z = np.empty(X1.shape)


for (i, j), val in np.ndenumerate(X1):
    x1 = val
    x2 = X2[i, j]
    p = clf.decision_function([[x1, x2]])
    Z[i, j] = p[0]
levels = [0.0, 0.0, 0.0]
linestyles = ['solid', 'solid', 'solid']
plt.contour(X1, X2, Z, levels,colors='black', linestyles=linestyles)
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Paired)

plt.axis('tight')
plt.show()

In [61]:
# libreria para clasificación por centroides cercanos 
from sklearn.neighbors import NearestCentroid

h = 0.2 #paso de la clasificación

# mapas de colores
cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA'])
cmap_bold = ListedColormap(['#FF0000', '#00FF00'])

for shrinkage in [None, 0.1]:
    # se crea clasificación de vecinos y se asigna el área
    clf = NearestCentroid(shrink_threshold=shrinkage)
    clf.fit(X, y)
    y_pred = clf.predict(X)
    print(shrinkage, np.mean(y == y_pred))

    # Se asignan las áreas dependiendo de la clasificacion de los puntos 
    # puntos en la malla [x_min, m_max]x[y_min, y_max].
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

    # Se muestra la grafica de las áreas
    Z = Z.reshape(xx.shape)
    plt.figure()
    plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

    # Se muestran los puntos de entrenamiento
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap_bold)
    plt.axis('tight')

plt.show()

(None, 0.79800000000000004)
(0.1, 0.76800000000000002)


In [62]:
from sklearn import neighbors, datasets

n_neighbors = 2
h = 0.1  # paso en la malla


# Mapas de colores
cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA'])
cmap_bold = ListedColormap(['#FF0000', '#00FF00'])

for weights in ['uniform', 'distance']:
    
    # Clasificación de vecinos cercanos.
    clf = neighbors.KNeighborsClassifier(n_neighbors, weights=weights)
    clf.fit(X, y)

    # Asignación del color de cada malla según el número de vecinos 
    # puntos en la malla [x_min, m_max]x[y_min, y_max].
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

    # Resultados en una malla de colores
    Z = Z.reshape(xx.shape)
    plt.figure()
    plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

    # Grafica de los puntos de entrenamiento. 
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap_bold)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.title("Clasificacion (k = %i, pesos = '%s')"
              % (n_neighbors, weights))

plt.show()